## 영화 5개 입력받기

In [ ]:
print("좋아하는 영화 5개를 입력해주세요(구분은 ,로): ")
movie_list = input().split(", ")
print(movie_list)

좋아하는 영화 5개를 입력해주세요(구분은 ,로): 
모가디슈, 블랙위도우, 라라랜드, 천문, 크루엘라
['모가디슈', '블랙위도우', '라라랜드', '천문', '크루엘라']


## 영화별 출연진 정보 가져오기

영화 목록에서 영화코드 추출

In [ ]:
import requests
import json

In [ ]:
movie_cd_list = []
director_list = []
selected_movie_list = []

for movie in movie_list:
  movie_cd_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=d99e474ec53733a372d8413bd2753506&movieNm='+movie
  res = requests.get(movie_cd_url)
  text = res.text
  d = json.loads(text)
  if d['movieListResult']['totCnt'] > 1:    # 동명의 영화가 있을 경우
    item_year = input(movie+"의 개봉연도를 입력하세요: ")   # 개봉연도로 구분
    for item in d['movieListResult']['movieList']:
      if item_year == item['openDt'][:4]:
          # director_list = director['peopleNm']
        movie_cd_list.append([item['movieCd'],
                              item['movieNm'],
                              item['openDt'][:4]])
        selected_movie_list.append(item['movieNm'])
  if d['movieListResult']['totCnt'] == 1:
    for item in d['movieListResult']['movieList']:
      movie_cd_list.append([item['movieCd'],
                            item['movieNm'],
                            item['openDt'][:4]])
      selected_movie_list.append(item['movieNm'])
  
print(movie_cd_list)
print(selected_movie_list)

모가디슈의 개봉연도를 입력하세요: 2021
블랙위도우의 개봉연도를 입력하세요: 2021
천문의 개봉연도를 입력하세요: 2019
[['20204117', '모가디슈', '2021'], ['20191951', '블랙 위도우', '2021'], ['20167904', '라라랜드', '2016'], ['20184571', '천문: 하늘에 묻는다', '2019'], ['20216362', '크루엘라', '2021']]
['모가디슈', '블랙 위도우', '라라랜드', '천문: 하늘에 묻는다', '크루엘라']


In [ ]:
import pandas as pd

movie_cdlist_df = pd.DataFrame(movie_cd_list)
movie_cdlist_df.columns = ['movieCd', 'movieNm','openYr']
movie_cdlist_df

,movieCd,movieNm,openYr
0,20204117,모가디슈,2021
1,20191951,블랙 위도우,2021
2,20167904,라라랜드,2016
3,20184571,천문: 하늘에 묻는다,2019
4,20216362,크루엘라,2021


영화코드로 배우추출

In [ ]:
movie_detail_list = []
movie_director_list = []
movie_cast_list = []    # 한 영화당 최대 10명의 배우를 추출하여 저장한 리스트
direct_list = []
movie_mainactor_list = []    # 한 영화당 최대 2명(주인공)의 배우를 추출하여 저장한 리스트
actor_list = []              # 5개 영화의 movie_cast_list을 모은 것
mainactor_list = []          # 5개 영화의 movie_mainactor_list을 모은 것
movie_actor_list = [[] for i in range(0,5)]
movie_direc_list = [[] for i in range(0,5)]
j=0

for movieCd in movie_cdlist_df['movieCd']:
  movie_director_list.clear()
  movie_cast_list.clear()
  movie_mainactor_list.clear()
  actor_list_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=d99e474ec53733a372d8413bd2753506&movieCd='+movieCd
  res = requests.get(actor_list_url)
  text = res.text
  d = json.loads(text)

  for direc in d['movieInfoResult']['movieInfo']['directors']:
    movie_director_list.append(direc['peopleNm'])
  direct_list+=movie_director_list

  for cast in d['movieInfoResult']['movieInfo']['actors']:
    if len(movie_mainactor_list) < 2:
      movie_mainactor_list.append(cast['peopleNm'])
    if len(movie_cast_list) <= 9:
      movie_cast_list.append(cast['peopleNm'])
  #print(movie_cast_list)
  actor_list += movie_cast_list
  mainactor_list += movie_mainactor_list

  j+=1

print(direct_list)
print(actor_list)
print(mainactor_list)

['류승완', '케이트 쇼트랜드', '데이미언 셔젤', '허진호', '크레이그 질레스피']
['김윤석', '조인성', '허준호', '구교환', '김소진', '정만식', '김재화', '박경혜', '박명신', '한철우', '스칼렛 요한슨', '플로렌스 퓨', '레이첼 와이즈', '데이빗 하버', '엠마 스톤', '라이언 고슬링', '핀 위트록', 'J.K. 시몬스', '로즈마리 드윗', '최민식', '한석규', '신구', '김홍파', '허준호', '김태우', '김원해', '임원희', '오광록', '박성훈', '엠마 스톤', '엠마 톰슨', '마크 스트롱']
['김윤석', '조인성', '스칼렛 요한슨', '플로렌스 퓨', '엠마 스톤', '라이언 고슬링', '최민식', '한석규', '엠마 스톤', '엠마 톰슨']


배우 등장 횟수 세기

In [ ]:
actor_dict = dict()
for i in actor_list:
  if i not in actor_dict:
    actor_dict[i]=1
  else:
    actor_dict[i] = actor_dict[i]+1

actor_dict

{'J.K. 시몬스': 1,
 '구교환': 1,
 '김소진': 1,
 '김원해': 1,
 '김윤석': 1,
 '김재화': 1,
 '김태우': 1,
 '김홍파': 1,
 '데이빗 하버': 1,
 '라이언 고슬링': 1,
 '레이첼 와이즈': 1,
 '로즈마리 드윗': 1,
 '마크 스트롱': 1,
 '박경혜': 1,
 '박명신': 1,
 '박성훈': 1,
 '스칼렛 요한슨': 1,
 '신구': 1,
 '엠마 스톤': 2,
 '엠마 톰슨': 1,
 '오광록': 1,
 '임원희': 1,
 '정만식': 1,
 '조인성': 1,
 '최민식': 1,
 '플로렌스 퓨': 1,
 '핀 위트록': 1,
 '한석규': 1,
 '한철우': 1,
 '허준호': 2}

주연배우(출연진리스트 맨 앞 2사람) 등장 횟수

In [ ]:
mainactor_dict = dict()
for i in mainactor_list:
  if i not in mainactor_dict:
    mainactor_dict[i]=1
  else:
    mainactor_dict[i] = mainactor_dict[i]+1

mainactor_dict

{'김윤석': 1,
 '라이언 고슬링': 1,
 '스칼렛 요한슨': 1,
 '엠마 스톤': 2,
 '엠마 톰슨': 1,
 '조인성': 1,
 '최민식': 1,
 '플로렌스 퓨': 1,
 '한석규': 1}

추천해줄 배우 5명 선별
* 2번 이상 등장한 배우 우선 선발
* 빈자리는 주연배우 리스트에서 랜덤으로 고름

In [ ]:
import random

prefer_actor_list = []
for key, value in actor_dict.items():
  if value > 1:
    prefer_actor_list.append(key)
if len(prefer_actor_list)<5:
  for i in range(5-len(prefer_actor_list)):
    actor = random.choice(mainactor_list)
    if actor not in prefer_actor_list:
      prefer_actor_list.append(actor)
  
prefer_actor_list

['허준호', '엠마 스톤', '최민식']

배우 코드 추출

In [ ]:
actor_code = []
filmo_list = []
final_filmo_list = []

for prefer_actor in prefer_actor_list:
  actor_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key=d99e474ec53733a372d8413bd2753506'+'&peopleNm='+prefer_actor
  res = requests.get(actor_list_url)
  text = res.text
  d = json.loads(text)

  for actor in d['peopleListResult']['peopleList']:
    final_filmo_list.clear()
    if actor['repRoleNm'] == '배우' and actor['peopleNmEn']!= "":
      filmo_list = actor['filmoNames'].split("|")
      for filmo in filmo_list:
        if filmo not in selected_movie_list:
          final_filmo_list.append(filmo)
      actor_code.append([actor['peopleCd'], actor['peopleNm'],actor['peopleNmEn'],
                         actor['repRoleNm'], final_filmo_list[:5]])

In [ ]:
print(selected_movie_list)
print(filmo_list)

['모가디슈', '블랙 위도우', '라라랜드', '천문: 하늘에 묻는다', '크루엘라']
['이상한 나라의 수학자', '천문: 하늘에 묻는다', '봉오동 전투', '헤븐: 행복의 나라로', '침묵', '특별시민', '대호', '루시', '명량', '신세계', '범죄와의 전쟁: 나쁜놈들 전성시대', '마당을 나온 암탉', '악마를 보았다', '악마를 보았다 감독판', '히말라야, 바람이 머무는 곳', '친절한 금자씨', '주먹이 운다(Crying Fist)', '꽃피는 봄이 오면', '올드보이', '오원 장승업 취화선', '취화선', '파이란', '해피엔드', '쉬리', '조용한 가족', '넘버 3', '사라는 유죄', '우리사랑 이대로', '우리들의 일그러진 영웅', '추락하는 것은 날개가 있다', '구로 아리랑', '겨울의 길목']


In [ ]:
actor_code_df = pd.DataFrame(actor_code)
actor_code_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm','filmoNames']
actor_code_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,10088453,허준호,HUH Joon-ho,배우,"[노량: 죽음의 바다, 결백, 퍼펙트맨, 국가부도의 날, 인랑]"
1,10049633,엠마 스톤,Emma Stone,배우,"[크루즈 패밀리: 뉴 에이지, 좀비랜드: 더블 탭, 더 페이버릿: 여왕의 여자, 빌..."
2,10072251,최민식,CHOI Min-shik,배우,"[이상한 나라의 수학자, 봉오동 전투, 헤븐: 행복의 나라로, 침묵, 특별시민]"
